In [1]:
from neo4j import GraphDatabase
 
# Define correct URI and AUTH arguments (no AUTH by default)
URI = "bolt://localhost:7687"
AUTH = ("", "")
 
with GraphDatabase.driver(URI, auth=AUTH) as client:
    # Check the connection
    client.verify_connectivity()
 
    # Create a user in the database
    records, summary, keys = client.execute_query(
        "CREATE (u:User {name: $name, password: $password}) RETURN u.name AS name;",
        name="John",
        password="pass",
        database_="memgraph",
    )
 
    # Get the result
    for record in records:
        print(record["name"])
 
    # Print the query counters
    print(summary.counters)
 
    # Find a user John in the database
    records, summary, keys = client.execute_query(
        "MATCH (u:User {name: $name}) RETURN u.name AS name",
        name="John",
        database_="memgraph",
    )
 
    # Get the result
    for record in records:
        print(record["name"])
 
    # Print the query
    print(summary.query)

John
SummaryCounters{labels_added: 1, nodes_created: 1, contains_updates: True, contains_system_updates: False}
John
MATCH (u:User {name: $name}) RETURN u.name AS name


In [57]:
import pandas as pd
import numpy as np
import random

size = 1000
base = pd.DataFrame({
    "id": np.random.randint(0,size,size=size),
    "status": random.choices(["active", "inactive"], weights=[.7,.3], k=size),
    "key": np.arange(size),
    # "diff": np.random.randint(0, 100, size=size)
})
base

,id,status,key
0,508,active,0
1,569,inactive,1
2,753,active,2
3,215,inactive,3
4,66,active,4
...,...,...,...
995,364,inactive,995
996,145,active,996
997,724,active,997
998,322,inactive,998


In [58]:
a = base.query("status=='inactive'")
a

,id,status,key
1,569,inactive,1
3,215,inactive,3
5,794,inactive,5
6,487,inactive,6
8,922,inactive,8
...,...,...,...
985,359,inactive,985
987,731,inactive,987
995,364,inactive,995
998,322,inactive,998


In [59]:
np.random.seed(55)

merged = pd.merge(left=a, right=base, on="id", how="left")
merged = merged.loc[merged['key_x']!=merged['key_y'],:]
merged['diff'] = np.random.randint(0, 100, size=merged.shape[0])
merged

,id,status_x,key_x,status_y,key_y,diff
2,215,inactive,3,active,446,77
4,794,inactive,5,active,615,26
5,794,inactive,5,active,827,39
7,487,inactive,6,active,466,72
12,504,inactive,15,inactive,503,29
...,...,...,...,...,...,...
663,731,inactive,987,inactive,820,9
665,364,inactive,995,inactive,53,49
666,364,inactive,995,inactive,177,96
668,322,inactive,998,inactive,282,79


In [60]:
nodes = pd.DataFrame({
    "nodes": base['key'],
    "status": base['status']
})
edges = pd.DataFrame({
    "source": merged['key_x'],
    "target": merged['key_y'],
    "diff": merged['diff']
})

In [61]:
(merged['key_x'] == merged['key_y']).sum()

np.int64(0)

In [62]:
(edges['source'] == edges['target']).sum()

np.int64(0)

In [63]:
from neo4j import GraphDatabase

URI = "bolt://localhost:7687"
AUTH = ("", "")

with GraphDatabase.driver(URI, auth=AUTH) as client:
    client.verify_connectivity()
    
    # Create nodes
    for _, row in nodes.iterrows():
        client.execute_query(
            "MERGE (n:Node {id: $id, status: $status})",
            id=row['nodes'], status=row['status'],
            database_="memgraph"
        )
    
    # Create edges
    # Create edges with diff property
    for _, row in edges.iterrows():
        client.execute_query(
            "MATCH (a:Node {id: $source}), (b:Node {id: $target}) CREATE (a)-[:CONNECTED {diff: $diff}]->(b)",
            source=int(row['source']), target=int(row['target']), diff=int(row['diff']),
            database_="memgraph"
        )


```cypher
MATCH path = (s)-[e]->(t)
WHERE e.diff < 50 AND s.status = "inactive"
RETURN path
ORDER BY e.diff ASC
```

```cypher
MATCH path = (s {status: "inactive"})-[*4..]->(t)
WHERE all(e in relationships(path) WHERE e.diff < 50)
RETURN path
```

In [54]:
edges.query("source==3")

,source,target,diff


In [55]:
edges

,source,target,diff
0,2,0,77
1,2,1,26
3,2,3,39
4,2,4,72
5,2,5,29
...,...,...,...
326994,995,994,16
326996,995,996,79
326997,995,997,96
326998,995,998,68
